<a href="https://colab.research.google.com/github/ghoshatanu857/Instrument_Automation/blob/main/Vector_Magnetometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
from plotly import graph_objs as go
import plotly.figure_factory as ff

In [ ]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 22,
            }
}

In [ ]:
def magField(current,distance):
  field = (4*np.pi*1e-7*current)/(2*np.pi*distance)
  return field

In [ ]:
def circle(inner_r,outer_r,circle_points,line_points):
  x_array=[]; y_array=[]
  x_array.append([points*np.cos(_points) for points,_points in zip(line_points,circle_points)])
  y_array.append([points*np.sin(_points) for points,_points in zip(line_points,circle_points)])
  return x_array, y_array

In [450]:
wire_radius = 25e-6; distance = 100e-6; wire_current = 10
linear_density = 10; init_angle = 0; final_angle = 2*np.pi
min_circular_density = 10; point_addition = 6

linear_points = np.linspace(wire_radius, distance, linear_density)
circular_points=[]; circular_angles=[]
[circular_points.append(int(min_circular_density + circular_density*point_addition)) for circular_density in range(linear_density)]
circular_points = np.array(circular_points)
if np.all(circular_points)==False:
  raise Exception('Check the values of circular_points')
[circular_angles.append(np.linspace(init_angle, final_angle,_points,endpoint=False)) for _points in circular_points]

mag_field = magField(current=wire_current,distance= linear_points)
x_points, y_points = circle(wire_radius, distance, circular_angles,line_points=linear_points)

In [451]:
circular_points

array([10, 16, 22, 28, 34, 40, 46, 52, 58, 64])

In [452]:
# def circle(inner_r,outer_r,circle_points,line_points):
#   x_array=[]; y_array=[]
#   x_array.append([points*np.cos(circle_points) for points in line_points])
#   y_array.append([points*np.sin(circle_points) for points in line_points])
#   return np.squeeze(np.array(x_array)), np.squeeze(np.array(y_array))

In [453]:
# wire_radius = 25e-6; distance = 100e-6; wire_current = 10
# linear_density = 10; circular_density = 20
# init_angle = -np.pi; final_angle = 0

# linear_points = np.linspace(wire_radius, distance, linear_density)
# circular_points = np.linspace(init_angle, final_angle, circular_density)

# mag_field = magField(current=wire_current,distance= linear_points)
# x_points, y_points = circle(wire_radius, distance, circular_points,line_points=mag_field)

In [454]:
linear_point = np.concatenate((-linear_points[::-1],linear_points))
X, Y = np.meshgrid(linear_point, linear_point)
radius = np.sqrt(X**2 + Y**2)

B = magField(current=wire_current, distance=radius)

In [457]:
# def polar_to_cartesian(radius, angle):
#   x = radius * np.cos(angle)
#   y = radius * np.sin(angle)
#   return np.squeeze(x), np.squeeze(y)

# r,theta=np.meshgrid(linear_points[-1],circular_angles[-1])
# x,y=polar_to_cartesian(r,theta)
# x,x_points[0][-1]

In [473]:
fig = go.Figure(data=go.Contour(x=linear_point,y=linear_point,z=B,contours_coloring='heatmap',
                                connectgaps=True,line_smoothing=1,colorbar=dict(title='Magnetic Field',titleside='right')))
fig.update_traces(line_width=0)

for radi in range(0,len(x_points[0]),2):
    x = x_points[0][radi]; y = y_points[0][radi]
    u = -y; v = x
    fig1 = ff.create_quiver(x, y, u, v, scale=125e-6*mag_field[radi]/linear_points[radi], arrow_scale=0.5, scaleratio=1.0, angle=np.pi/9, line=dict(width=1.2, color='black'))
    fig.add_traces(data=fig1.data)

fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers', marker_size=15, marker=dict(color='yellow')))
fig.add_trace(go.Scatter(x=linear_point, y=-wire_radius*np.ones(linear_point.shape[0]), mode='lines', marker=dict(color='yellow'),fill=None))
fig.add_trace(go.Scatter(x=linear_point, y=-distance*np.ones(linear_point.shape[0]), mode='lines', marker=dict(color='grey'), fill='tonexty'))

fig.update_layout(showlegend=False)
fig.update_layout(template=fig_template, title = 'Magnetic Field vs Radial Distance',width=800, height=600)
fig.update_xaxes(range = [-distance,distance]); fig.update_yaxes(range = [-distance,distance])
fig.show()

In [ ]:
# fig = go.Figure()

# x=x_points;y=y_points; u=-y[::-1];v=x[::-1]
# fig1 = ff.create_quiver(x, y, u, v, scale=.1, arrow_scale=0.2,
#                       scaleratio=0.7,angle=np.pi/9,line=dict(width=1.5,color='black' ))
# fig.add_trace(go.Scatter(x = [0], y = [0], mode = 'markers', marker_size = 10))
# fig.update_layout(template = fig_template,width=800,height=600)
# fig.show()